# Проект для Викишоп с BERT

# Table of Contents
* [Описание проекта](#0.Описание)
    * [Описание проекта](#0.1.Цель)
    * [План работ](#0.2.План)
    * [Примечания к плану работ](#0.3.Прим)
    * [Данные](#0.3.Данные)
* [1. Загрузка и подготовка данных](#1.Глава1)
    * [1.1. Импорт библиотек](#1.1.Импорт_библиотек)
    * [1.2. Определение констант](#1.2.Опред_констант)
    * [1.3. Загрузка данных](#1.3.Загрузка_данных)
* [2. Подготовка признаков с BERT](#2.Подг_наборов)
    * [2.1. Токенизация и инициализация pretrained модели](#2.1.Токенизация)
    * [2.2. Разделение train, test ](#2.2.Таргет_Призн)
* [3. Обучение моделей](#3.Обучение_мод)
    * [3.1. Линейная регрессия](#3.1.LR)
    * [3.2. Финальное тестирование](#3.2)
* [4. Вывод](#4.Вывод)


# Описание проекта <a class="anchor" id="0.Описание"></a>

## Описание проекта <a class="anchor" id="0.1.Цель"></a>

1. Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 
2. Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.
3. Решить задачу можно как с помощью BERT, так и без этой нейронки. Если хотите попробовать BERT — Выполните проект локально. Упомяните BERT в заголовке проекта в первой ячейке.

## План работ <a class="anchor" id="0.2.План"></a>

1. Загрузите данные
2. Проанализируйте данные.
3. Обучите разные модели с различными гиперпараметрами. Сделайте тестовую выборку размером 10% от исходных данных.
4. Проверьте данные на тестовой выборке и сделайте выводы.

# 1. Загрузка и подготовка данных <a class="anchor" id="1.Глава1"></a>


## 1.1 Импорт библиотек <a class="anchor" id="1.1.Импорт_библиотек"></a>

In [7]:
#загрузим все необходимые библиотеки и инструменты
import torch
import transformers as ppb
from transformers import BertConfig
from transformers import BertModel
from transformers import BertTokenizer 
from transformers import BertForTokenClassification
from transformers import AutoModelForMaskedLM
#from transformers import BERT

#from transformers.utils import send_example_telemetry
from tqdm import notebook

import pandas as pd
import numpy as np

from sklearn.utils import shuffle

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid
from imblearn.over_sampling import SMOTE


## 1.2 Определение констант <a class="anchor" id="1.2.Опред_констант"></a>

In [111]:

RANDOM_STATE = 27182
state = np.random.RandomState(RANDOM_STATE)


## 1.3 Загрузка и проверка данных <a class="anchor" id="1.3.Загрузка_данных"></a>

1. Загружаем данные.
2. Проверяем на NaN, мультиколинерность, дублирование категориальных признаков, лишние данные/столбцы.
3. Убираем дублирование.

In [3]:

try:
    #df = pd.read_csv(r'c:\%Users%\datasets\toxic_comments.csv')  r'c:\Users\e_rotar\Documents\yp\pr13\datasets\toxic_comments.csv' 
    df = pd.read_csv(r'c:\%Users%\toxic_comments.csv')
    print('Загрузился Path Windows!')
except Exception:
    print('Path Windows не загрузился!')
    
try:
    df = pd.read_csv(r'/datasets/toxic_comments.csv')
    print('Загрузился Path Yandex!')

except Exception:
    print('Path Yandex не загрузился!')    


Загрузился Path Windows!
Path Yandex не загрузился!


In [113]:

def about_df(df,sample_size=5, graph = True, categorical_ = False):
    print(f'Первые {sample_size} строк')
    display(df.head(sample_size))
    print(f'Последние {sample_size} строк')
    display(df.tail(sample_size))
    print(f'\n Основная информация')
    print(df.info())
    display(df.describe(include='all'))
    display(
        pd.DataFrame(
            np.array([df.isna().sum(), df.isna().mean()]).T,
            columns = ['кол-во пропусков','доля пропусков'],
            index=df.columns
        ).style.background_gradient('coolwarm')
    )
    
    print(f'\n Кол-во и доля дубликатов')
    
    display( df.duplicated().head() )
    print(1 - df.duplicated().value_counts()/len(df) )

1. Проверяем df
2. Функция проверки NaN, мультиколинерность, дублирование категориальных признаков, лишние данные/столбцы.

In [48]:

about_df(df)


Первые 5 строк


,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


Последние 5 строк


,Unnamed: 0,text,toxic
159287,159446,""":::::And for the second time of asking, when ...",0
159288,159447,You should be ashamed of yourself \n\nThat is ...,0
159289,159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,159449,And it looks like it was actually you who put ...,0
159291,159450,"""\nAnd ... I really don't think you understand...",0



 Основная информация
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB
None


,Unnamed: 0,text,toxic
count,159292.000000,159292,159292.000000
unique,NaN,159292,NaN
top,NaN,Explanation\nWhy the edits made under my usern...,NaN
freq,NaN,1,NaN
mean,79725.697242,NaN,0.101612
std,46028.837471,NaN,0.302139
min,0.000000,NaN,0.000000
25%,39872.750000,NaN,0.000000
50%,79721.500000,NaN,0.000000
75%,119573.250000,NaN,0.000000


,кол-во пропусков,доля пропусков
Unnamed: 0,0.000000,0.000000
text,0.000000,0.000000
toxic,0.000000,0.000000



 Кол-во и доля дубликатов


0    False
1    False
2    False
3    False
4    False
dtype: bool

False    0.0
dtype: float64


1. Проверил элементы столбца 'Unnamed: 0' с индексами (есть несовпадения с индекса 6080).
2. Вероятно несовпадения случайные. Не уведел двух строк в одной. Столбец 'Unnamed: 0' не несет дополнительную информацию.
3. Считаю столбец 'Unnamed: 0' ошибочным и удаляю его


In [5]:
#print(df[df['Unnamed: 0'] != df.index].head())
#print(df.loc[159291,'Unnamed: 0'])
err_txt=[]

for i in range(6079,159291):
    if ((df.loc[i-1,'Unnamed: 0']+1) !=df.loc[i,'Unnamed: 0']):
        err_txt.append([df.loc[i,"text"]])
            
print(err_txt[0:4])

[['"::I\'ll alos be looking in to see how this is going, as GRC is a big deal around these parts. Seek his grace \n\n"'], ["Active Members article \n\nHello. I don't think it is appropriate to add e-mail addresses or such information to a Wikipedia article, especially if it has no context. Will probably be deleted."], ['What is ALRs problem. Can someone please investiagate and sanction this user. Truthseekers666 (talk) Truthseekers666 Matthew Williams 2-2010   """'], ["Cubits, hogsheads, rods, chains, etc? \n\nSince we've got to have Imperial as well as Metric, maybe we should be adding all the above (and more)?  Jimbo must be off his nut..."]]


1. 'Unnamed: 0' не несет никакой информации.
2. Убираем


In [10]:
df = df.drop('Unnamed: 0', axis=1)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB
None


Проверил по балансу toxic:
1. Данные не сбалансированы, токсичных коментариев только 10%
2. После подготовки данных проведем upsampling


In [11]:
print(df['toxic'].mean())


0.10161213369158527


1. Уменшил выборку до 1%.

In [12]:
print(df['toxic'].mean())
df_ = df.sample(frac=0.01, random_state=RANDOM_STATE)
print(df_['toxic'].mean())
print(df_.shape)

0.10161213369158527
0.10860012554927809
(1593, 2)


# 2. Подготовка признаков с BERT <a class="anchor" id="2.Подг_наборов"></a>

## 2.1. Токенизация и инициализация pretrained модели <a class="anchor" id="2.1.Токенизация"></a>

1. Выполним токенизацию подготовленной моделью - BertTokenizer.from_pretrained
2. Выполним классификацию токенов подготовленной моделью
3. DistilBertModel загрузить корректно не получилось


1. Выполним энкодинг c помощью BERT.
2. В результате использовался большой словарь токенов BertModel под 30 тыс. слов.
3. Пришлось ограничить количество столбцов/векторов до 512. Модель создавалась для более мощных систем чем домашний комп.
4. Применен метод padding, что-бы векторы в каждом тексте были равны.
5. Подготовлена маска внимания/attention_mask

In [63]:
%%time

tokenized = df_['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512, padding=True, truncation=True))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)
config = ppb.BertConfig.from_pretrained('unitary/toxic-bert')

Wall time: 5.18 s


In [52]:
print(ppb.__version__)


4.26.1


In [12]:
print(max_len)

512


1. Распечатал токенайзер, что-бы иметь представление о модели.

In [62]:
%%time
print(tokenizer)

BertTokenizer(name_or_path='unitary/toxic-bert', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})
Wall time: 1 ms


1. Токенизированная матрица

In [19]:
%%time
print(tokenized)
print(tokenized.shape)
#print(df_ups.shape)

26447     [101, 2004, 4842, 15776, 1998, 1040, 7274, 144...
155507    [101, 1038, 2094, 5443, 10930, 2497, 1998, 109...
22522     [101, 2017, 2031, 2042, 8184, 8534, 2013, 9260...
36661     [101, 2065, 2115, 2063, 1996, 5310, 2071, 2017...
98165     [101, 3087, 4699, 1999, 3773, 1996, 2197, 2544...
                                ...                        
147642    [101, 2293, 2023, 28516, 1045, 2074, 2293, 202...
52296     [101, 8756, 3931, 2128, 15773, 2044, 1037, 271...
141471    [101, 1045, 2123, 2102, 2903, 2008, 1996, 1276...
5317      [101, 2174, 2017, 2134, 2102, 3696, 2115, 1008...
26817             [101, 5314, 1998, 2417, 7442, 10985, 102]
Name: text, Length: 319, dtype: object
(319,)
Wall time: 6.99 ms


In [64]:
print(tokenized.shape)
print(padded.shape)
print(attention_mask.shape)

(1593,)
(1593, 512)
(1593, 512)


In [65]:
print(tokenized.head())
print(padded[0:4,:])

print(attention_mask[0:4,:])

26447     [101, 2004, 4842, 15776, 1998, 1040, 7274, 144...
155507    [101, 1038, 2094, 5443, 10930, 2497, 1998, 109...
22522     [101, 2017, 2031, 2042, 8184, 8534, 2013, 9260...
36661     [101, 2065, 2017, 1005, 2128, 1996, 5310, 1010...
98165     [101, 3087, 4699, 1999, 3773, 1996, 2197, 2544...
Name: text, dtype: object
[[ 101 2004 4842 ...    0    0    0]
 [ 101 1038 2094 ...    0    0    0]
 [ 101 2017 2031 ...    0    0    0]
 [ 101 2065 2017 ...    0    0    0]]
[[1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]]


1. Самая длительная процедура. Оптимизация векторов.
2. На ноуте 330 значений оптимизировалось 5 часов (оставил вычисления с ноута, 330 строк 5 часов 13 мин.
3. На старой станции, 3000 значений оптимизирует 40мин.

In [20]:
%%time
batch_size = 10
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/31 [00:00<?, ?it/s]

Wall time: 5h 13min 13s


1. Получим матрицы признаков и таргетов.
2. 9 строк признаков не вошло в эмбединг, уберем эти строки и из таргетов.
3. Тут оставил результаты расчета на ноуте, на нем пишу отчет.
4. Для меня загадка, почему схлопнулось до 6 столбцов.

In [32]:
X_ = np.concatenate(embeddings)
print(X_.shape)
y_ = df_['toxic'][0:310]
print(sum(y_))
y_np = y_.values
print(y_.shape)

(310, 6)
43
(310,)


In [18]:

X_df = pd.DataFrame(data=X_)
display(X_df.head())
display(X_df.tail())
y_df = pd.Series(data=y_np)
X_df.to_csv(r'c:\%Users%\datasets\X_df.csv')  
y_df.to_csv(r'c:\%Users%\datasets\y_df.csv')  
print(y_df.head())
print(y_df.tail())


'print(y_df.head())\nprint(y_df.tail())\n'

1. Импортировал значения станции
2. Как никак 1580 значений


In [114]:
X_df = pd.read_csv(r'c:\%Users%\datasets\X_df1.csv',index_col=[0])  
y_df = pd.read_csv(r'c:\%Users%\datasets\y_df1.csv',index_col=[0])  
print(X_df.head())
print(X_df.tail())
print(y_df.head())
print(y_df.tail())

          0         1         2         3         4         5
0  0.221975  0.645929  0.120793  0.395917  0.027843  1.044683
1  0.238338  0.547478  0.241995  0.466450  0.031583  1.018048
2  0.273470  0.469837  0.228394  0.515334 -0.023717  1.003967
3  0.231933  0.520798  0.254784  0.476044 -0.080721  0.982087
4  0.196502  0.582510  0.103276  0.373847  0.028609  0.884021
             0         1         2         3         4         5
1575  0.106456  0.596228  0.093451  0.359686  0.057490  1.001742
1576  0.249963  0.454072  0.149404  0.359429 -0.024041  0.918310
1577  0.221888 -0.177940 -0.009043 -0.106122  0.102372  0.208950
1578  0.185396  0.538506  0.247365  0.517950  0.000176  0.999980
1579  0.225921  0.695862  0.139792  0.463811  0.131721  1.012390
   0
0  0
1  0
2  0
3  0
4  0
      0
1575  0
1576  0
1577  1
1578  0
1579  0


## 2.2. Разделение наборов на train, test, таргеты и признаки <a class="anchor" id="2.2.Таргет_Призн"></a>

In [122]:
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2)
print('train')
print(X_train.shape)
print(y_train.shape)
print(X_train.head())
print(X_train.tail())
print(y_train.head())
print(y_train.tail())

print('test')
print(X_test.shape)
print(y_test.shape)
print(X_test.head())
print(X_test.tail())
print(y_test.head())
print(y_test.tail())


train
(1264, 6)
(1264, 1)
             0         1         2         3         4         5
243   0.273652  0.448398  0.154735  0.467910 -0.005949  0.962566
137   0.233285  0.550384  0.378357  0.539977 -0.034335  1.011030
36    0.214641  0.591989  0.134738  0.442989  0.014816  1.108101
1511  0.193792  0.564257  0.211581  0.420615  0.003101  1.012922
189   0.213057  0.293138  0.249988  0.431603 -0.129562  0.916270
             0         1         2         3         4         5
594   0.135012  0.536017  0.113799  0.427400  0.011443  1.092909
726   0.210975  0.580792  0.193263  0.357734  0.057507  0.991030
970   0.240265  0.388861  0.216818  0.434109 -0.005781  1.044691
1425  0.262905  0.428502  0.246064  0.504907 -0.043291  1.029067
1567  0.215138  0.485129  0.228926  0.423884  0.005770  1.052290
      0
243   0
137   0
36    0
1511  0
189   0
      0
594   0
726   0
970   0
1425  0
1567  0
test
(316, 6)
(316, 1)
             0         1         2         3         4         5
161   0.52

1. Хотя, предварительные оценки показали, и так все неплохо получается.
2. Правильно будет устранить дисбаланс в таргетах.


In [123]:
print(y_train.mean())


0    0.106804
dtype: float64


# 3. Обучение и  тестирование моделей <a class="anchor" id="3.Обучение_мод"></a>

## 3.1. Сложная валидация с Ucampling <a class="anchor" id="3.1.LR"></a>

1. После upsampling делать валидаци некоректно, поэтому стандартная процедура скоректирована.
2. Вначале отделил test/train
3. Задаю модель
4. На внешнем цикле меняю гиперпараметры
5. Потом делю выборку train на фолды  с помощью KFold в каждом фолде 4/5 это train, а 1/5 это valid. 
6. Провожу Upsamling SMOT (необходимо инсталировать в Conda или в Colab). У меня pip install не работал корректно в Conde.
7. Обучаю на X_train_fold_upsample, y_train_fold_upsample
8. Проверяю на валидность, выборка valid не сэмплированная. 
9. Нахожу метрику score = f1_score(y_val_fold, model_obj.predict(X_val_fold))
9. Собрал результаты в многомерный список, перевел в Датафрейм
10. Сделал scores_df.groupby('C', as_index=False).mean()
11. Результаты почти плоские. Выделил как оптимальный C=1
12. Проведу тест на тестовой выборке, с С=1


In [127]:
%%time

lr= LogisticRegression()
param={"C" :[0.0625, 0.125,0.25,0.5,1,2, 4],
        "class_weight" : ['balanced'],
        "solver":['liblinear']}
model = lr
kf = KFold(n_splits=5)
scores = []
#kv = KFold(n_splits=5, random_state=None, shuffle=False)

for g in ParameterGrid(param):
    model.set_params(**g)
#    print(model)
#    print(g)
#    print(**g)
#    print(X_train)
#    print(y_train)
#    print(kf)
    
    for i, (train_fold_index, val_fold_index) in enumerate(kf.split(X_train)):
        #print(f"Fold {i}:")
        #print(f"  Train: index={train_fold_index}")
        #print(f"  Test:  index={val_fold_index}")

        X_train_fold, y_train_fold = X_train.iloc[train_fold_index], y_train.iloc[train_fold_index]
        # Get the validation data
        X_val_fold, y_val_fold = X_train.iloc[val_fold_index], y_train.iloc[val_fold_index]

        # Upsample only the data in the training section
        X_train_fold_upsample, y_train_fold_upsample = smoter.fit_resample(X_train_fold,
                                                                           y_train_fold)
        print("Средний таргет после крутого апсемпла:", y_train_fold_upsample.mean())
        # Fit the model on the upsampled training data
#        print('X_train_fold_upsample',X_train_fold_upsample.shape)
#        print('y_train_fold_upsample',y_train_fold_upsample.shape)
#        print('g',g)
        model_obj = model.fit(X_train_fold_upsample, y_train_fold_upsample.values.ravel())
        # Score the model on the (non-upsampled) validation data

        score = f1_score(y_val_fold, model_obj.predict(X_val_fold))
        scores.append([g["C"],i,score])
scores_df = pd.DataFrame(data = scores, columns = ['C', 'Fold', 'f1_score'])

display(scores_df.groupby('C', as_index="C").mean())

Средний таргет после крутого апсемпла: 0    0.5
dtype: float64
Средний таргет после крутого апсемпла: 0    0.5
dtype: float64
Средний таргет после крутого апсемпла: 0    0.5
dtype: float64
Средний таргет после крутого апсемпла: 0    0.5
dtype: float64
Средний таргет после крутого апсемпла: 0    0.5
dtype: float64
Средний таргет после крутого апсемпла: 0    0.5
dtype: float64
Средний таргет после крутого апсемпла: 0    0.5
dtype: float64
Средний таргет после крутого апсемпла: 0    0.5
dtype: float64
Средний таргет после крутого апсемпла: 0    0.5
dtype: float64
Средний таргет после крутого апсемпла: 0    0.5
dtype: float64
Средний таргет после крутого апсемпла: 0    0.5
dtype: float64
Средний таргет после крутого апсемпла: 0    0.5
dtype: float64
Средний таргет после крутого апсемпла: 0    0.5
dtype: float64
Средний таргет после крутого апсемпла: 0    0.5
dtype: float64
Средний таргет после крутого апсемпла: 0    0.5
dtype: float64
Средний таргет после крутого апсемпла: 0    0.5
dtype: 

,Fold,f1_score
C,,
0.0625,2.0,0.853611
0.1250,2.0,0.853575
0.2500,2.0,0.853389
0.5000,2.0,0.856984
1.0000,2.0,0.859106
2.0000,2.0,0.855433
4.0000,2.0,0.855726


Wall time: 758 ms


## 3.2. Финальное тестирование <a class="anchor" id="3.2"></a>

In [132]:
%%time

lr= LogisticRegression()
param={"C" :1,
        "class_weight" : 'balanced',
        "solver":'liblinear'}
model = lr
model.set_params(**param)
X_train_fold_upsample, y_train_fold_upsample = smoter.fit_resample(X_train, y_train)
model_obj = model.fit(X_train_fold_upsample, y_train_fold_upsample.values.ravel())
# Score the model on the (non-upsampled) validation data

score = f1_score(y_test, model_obj.predict(X_test))
print('f1_score_test:', score)

f1_score_test: 0.8314606741573033
Wall time: 25 ms


# 4. Вывод <a class="anchor" id="4.Вывод"></a>

1. Задача оценки токсичных коментариев сделана с использованием предтестированных моделей BERT.
2. Данные предварительно очищены. Выявлен дисбаланс по таргетам. Оставлять его нельзя.
3. Токенизация, эмбединг и padding выполнены с помощью BERT.
4. Самая длительная процедура - оптимизация векторов признаков, после padding. Выполняется достаточно долго - на ноуте получилось 5 часов 300 твитов. На системном блоке 40 минут, 1600 твитов.
5. После оптимизации, получилось всего 6 признаков. 
6. Было выполнено разделение target и признаки, train и test.
7. Затем был проведен Upsampling.
7. Для предсказаний использовалась линейная регрессия. Результат хороший:
        A. f1 на валидации 85%,
        B. f1 на test 83%.
